In [ ]:
from pathlib import Path
import glob

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import xarray as xr
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
model_df = pd.read_csv("../data/processed/model_inputs.csv")

In [ ]:
list(model_df)

In [ ]:
input_df = model_df[
    [
        "lat_2",
        "lon_2",
        "time",
        "aet",
        "def",
        "pdsi",
        "pet",
        "pr",
        "srad",
        "ro",
        "soil",
        "swe",
        "precip",
        "lwe_thickness",
    ]
]
input_df["precip"] = input_df["precip"].fillna(0)

In [ ]:
input_df = input_df.dropna().reset_index(drop=True)

In [ ]:
input_df

In [ ]:
X_df = input_df.iloc[:, 0:13]

In [ ]:
y_df = input_df.iloc[:, -1]

In [ ]:
time_dict = dict(zip(X_df["time"].unique().tolist(), range(0, X_df.time.nunique())))

In [ ]:
X_df["time"] = X_df.time.map(time_dict)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_df.to_numpy(), y_df.to_numpy(), test_size=0.2
)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)

In [ ]:
n_features = X_train.shape[1]

In [ ]:
# Setting up error metrics
rmse = tf.keras.metrics.RootMeanSquaredError(name="root_mean_squared_error", dtype=None)

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
model = MLPRegressor(hidden_layer_sizes=(24, 18, 12, 6,4,), batch_size=len(X_df[X_df.time == 0]), solver="lbfgs", alpha=0.001)

In [ ]:
model.fit(X_train_scaled, y_train)

In [ ]:
error = model.evaluate(X_train_scaled, y_train, verbose=0)

In [ ]:
error

In [ ]:
X_df[X_df.time == 1]

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
from sklearn import metrics

print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
metrics.r2_score(y_test, y_pred)

In [ ]:
import pickle
filename = '../models/mlp_model.sav'

In [ ]:
pickle.dump(model, open(filename, 'wb')) 

In [ ]:
model = pickle.load(open(filename, "rb"))

In [ ]:
time_dict

In [ ]:
def predict_ts(ts_file):
    ts_df = pd.read_parquet(ts_file)
    ts_df["precip"] = ts_df["precip"].fillna(0)
    ts_df["time"] = ts_df.time.dt.strftime("%Y-%m-%d").map(time_dict)
    ts_df = ts_df.dropna().reset_index(drop=True)
    if len(ts_df) > 0:
        ts_test_x = ts_df.to_numpy()
        ts_step_scaled = scaler.transform(ts_test_x)
        ts_pred = model.predict(ts_step_scaled)
        output_df = pd.DataFrame(ts_pred, columns=["lwe_thickness"])
        final_ts_df = pd.concat([ts_df[["lat", "lon", "time"]], output_df], axis=1)
        final_ts_df["time"] = final_ts_df["time"].map(
            dict(zip(time_dict.values(), time_dict.keys()))
        )
        final_ts_df["time"] = pd.to_datetime(final_ts_df.time)
        ds = final_ts_df.groupby(["lat", "lon", "time"]).mean().to_xarray()
        return ds

In [ ]:
xr.concat(
    [
        predict_ts(_file)
        for _file in sorted(glob.glob("../data/processed/target_grid/*")) if Path(_file).name in time_dict
    ],
    dim="time",
).to_netcdf("../data/processed/mlp_output.nc")

In [ ]:
predicted_grace = xr.open_dataset("../data/processed/mlp_output.nc")

In [ ]:
actual_grace = xr.open_dataset("../data/processed/grace.nc")

In [ ]:
actual_grace

In [ ]:
predicted_grace

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 10))
predicted_grace.lwe_thickness.sel(time="2008-01-01").plot(ax=ax[1])
actual_grace.lwe_thickness.sel(time="2008-01-01").plot(ax=ax[0])
# plt.savefig("../reports/figures/actual_vs_dl.png")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 10))
predicted_grace.lwe_thickness.sel(time="2008-01-01").plot(ax=ax[1])
actual_grace.lwe_thickness.sel(time="2008-01-01").plot(ax=ax[0])
plt.savefig("../reports/figures/actual_vs_mlp.png")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(25, 10))
predicted_grace.lwe_thickness.mean(["lat", "lon"]).plot(ax=ax, label="Predicted")
actual_grace.lwe_thickness.mean(["lat", "lon"]).plot(ax=ax, label="Original")
plt.legend()
# plt.savefig("mean.png")